Loads data saved from notebook 01_load_and_clean

In [36]:
import pandas as pd
# Load the processed Spotify UK dataset
uk = pd.read_csv("../data/processed/spotify_uk.csv")
uk["date"] = pd.to_datetime(uk["date"])
uk.head()
len(uk)

455308

Sort the sataset, to caluculate trends and days-on-chart, we must ensure rows are in the correct order.

In [37]:
# Convert the 'date' column to datetime format and sort the DataFrame
uk["date"] = pd.to_datetime(uk["date"])
# Sort by 'title' and 'date' columns
uk = uk.sort_values(["title", "date"]).reset_index(drop=True)


Feature 1: How long has a song been on the chart?

In [38]:
uk["days_on_chart"] = uk.groupby("title").cumcount() + 1
uk[["title", "date", "days_on_chart"]].head(10)

,title,date,days_on_chart
0,#AYFKM (Freestyle),2019-09-02,1
1,#AYFKM (Freestyle),2019-09-03,2
2,#AYFKM (Freestyle),2019-09-04,3
3,#BrooklynBloodPop!,2021-05-14,1
4,#BrooklynBloodPop!,2021-05-15,2
5,#BrooklynBloodPop!,2021-05-16,3
6,#BrooklynBloodPop!,2021-05-17,4
7,#BrooklynBloodPop!,2021-05-18,5
8,#BrooklynBloodPop!,2021-05-19,6
9,#BrooklynBloodPop!,2021-05-20,7


For each song, count how many days it appeared before, 1 first appearance, 2 seconde ...

Feature 2: Did the song move up, down or stayed in the same possition?

In [39]:
# Calculate the trend of each song's position on the chart
trend_map = {
    "MOVE_UP": 1,
    "MOVE_DOWN": -1,
    "SAME_POSITION": 0
}
# Map the trend values to numerical values
uk["trend_num"] = uk["trend"].map(trend_map)
uk[["trend", "trend_num"]].head()

,trend,trend_num
0,NEW_ENTRY,NaN
1,MOVE_UP,1.0
2,MOVE_UP,1.0
3,NEW_ENTRY,NaN
4,SAME_POSITION,0.0


Feature 3: Where did the song rank yesterday?

In [40]:
uk["prev_rank"] = uk.groupby("title")["rank"].shift(1)

Feature 4: How fast is the song moving?

In [41]:
# Calculate difference between current rank and previous rank
uk["rank_change"] = uk["prev_rank"] - uk["rank"]

Feature 5: Normalization of large differences in stream numbers.

In [42]:
import numpy as np
# Minimize stream numbers to let ML model handle data better
uk["log_streams"] = np.log1p(uk["streams"])
uk[["streams", "log_streams"]].head()


,streams,log_streams
0,21858.0,9.992368
1,23359.0,10.058780
2,23800.0,10.077483
3,NaN,NaN
4,NaN,NaN


Save the processed dataset

In [43]:
# Save the updated DataFrame with new features
uk.to_csv("../data/processed/spotify_uk_features.csv", index=False)
